## **Get schema**

Dado que la cantidad de columnas y sus nombres varían entre archivos, en este notebook se procede a interactuar con estos archivos para obtener los nombres de las columnas de interés para un posterior procesamiento

In [27]:
!ls -sh | grep recorridos-realizados

668K recorridos-realizados-2010.csv
 83M recorridos-realizados-2011.csv
139M recorridos-realizados-2012.csv
247M recorridos-realizados-2013.csv
424M recorridos-realizados-2014.csv
112M recorridos-realizados-2015.csv
133M recorridos-realizados-2016.csv
238M recorridos-realizados-2017.csv
630M recorridos-realizados-2018.csv
649M recorridos-realizados-2019.csv


In [28]:
!head -1 recorridos-realizados-201*.csv

==> recorridos-realizados-2010.csv <==
periodo,fecha_origen_recorrido,id_estacion_origen,nombre_estacion_origen,long_estacion_origen,lat_estacion_origen,domicilio_estacion_origen,duracion_recorrido,fecha_destino_recorrido,id_estacion_destino,nombre_estacion_destino,long_estacion_destino,lat_estacion_destino,domicilio_estacion_destino

==> recorridos-realizados-2011.csv <==
periodo,fecha_origen_recorrido,id_estacion_origen,nombre_estacion_origen,long_estacion_origen,lat_estacion_origen,domicilio_estacion_origen,duracion_recorrido,fecha_destino_recorrido,id_estacion_destino,nombre_estacion_destino,long_estacion_destino,lat_estacion_destino,domicilio_estacion_destino

==> recorridos-realizados-2012.csv <==
periodo,fecha_origen_recorrido,id_estacion_origen,nombre_estacion_origen,lat_estacion_origen,long_estacion_origen,domicilio_estacion_origen,duracion_recorrido,fecha_destino_recorrido,id_estacion_destino,nombre_estacion_destino,lat_estacion_destino,long_estacion_destino,domicilio_estacio

Luego de un vistazo rápido a las columnas presentes en cada archivo, concluyo que dado para un recorrido es necesario tener el identicador de la estación origen y destino. Luego mediante un cruzamento con otro archivo sobre las estaciones, se podrán obtener datos como el nombre de la estación. 
Los campos sobre la longitud y latitud de la estación ya son útiles para luego obtener, si es necesario, la dirección concreta sobre la que se ubica la estación.
Obviamente, es necesario retener los campos sobre las fechas y duración del recorrido.
Los demás campos se pueden descartar, como datos concretos sobre la capacidad o los usuarios del recorrido (no todos los archivos tienen datos sobre el usuario del recorrido).

In [29]:
import pandas as pd
import json

In [30]:
files = [f"recorridos-realizados-201{i}.csv" for i in range(10)]
files

['recorridos-realizados-2010.csv',
 'recorridos-realizados-2011.csv',
 'recorridos-realizados-2012.csv',
 'recorridos-realizados-2013.csv',
 'recorridos-realizados-2014.csv',
 'recorridos-realizados-2015.csv',
 'recorridos-realizados-2016.csv',
 'recorridos-realizados-2017.csv',
 'recorridos-realizados-2018.csv',
 'recorridos-realizados-2019.csv']

Crearemos un diccionario, donde cada clave representará el nombre del archivo y su respectivo valor será una lista con sus columnas

In [31]:
columns_per_files = {}
for file in files:
    df = pd.read_csv(file, nrows=1)
    print(f"File: {file}\tColumns: {len(df.columns)}\n")
    columns_per_files[file] = df.columns

File: recorridos-realizados-2010.csv	Columns: 14

File: recorridos-realizados-2011.csv	Columns: 14

File: recorridos-realizados-2012.csv	Columns: 14

File: recorridos-realizados-2013.csv	Columns: 15

File: recorridos-realizados-2014.csv	Columns: 20

File: recorridos-realizados-2015.csv	Columns: 15

File: recorridos-realizados-2016.csv	Columns: 15

File: recorridos-realizados-2017.csv	Columns: 15

File: recorridos-realizados-2018.csv	Columns: 15

File: recorridos-realizados-2019.csv	Columns: 19



Con los campos que todos los archivos deben tener, procedemos a filtrar.
En algunos archivos difiere el orden de las columnas. Unos presentan latidad y longitud, mientras que otros lo presentan en el orden inverso. También ocurre que para un mismo campo, presentan un nombre diferente como "fecha_origen_recorrido" y "origen_recorrido_fecha"

In [32]:
for file, columns in columns_per_files.items():
    columns_per_files[file] = list(filter(lambda s: \
                                         'fecha' in s or 'id_estacion' in s \
                                         or 'lat' in s or 'long' in s or 'duracion' in s, \
                                         columns))
    print(f"File: {file}\t Columns:{len(columns_per_files[file])}")
    print(columns, end="\n\n")

File: recorridos-realizados-2010.csv	 Columns:9
Index(['periodo', 'fecha_origen_recorrido', 'id_estacion_origen',
       'nombre_estacion_origen', 'long_estacion_origen', 'lat_estacion_origen',
       'domicilio_estacion_origen', 'duracion_recorrido',
       'fecha_destino_recorrido', 'id_estacion_destino',
       'nombre_estacion_destino', 'long_estacion_destino',
       'lat_estacion_destino', 'domicilio_estacion_destino'],
      dtype='object')

File: recorridos-realizados-2011.csv	 Columns:9
Index(['periodo', 'fecha_origen_recorrido', 'id_estacion_origen',
       'nombre_estacion_origen', 'long_estacion_origen', 'lat_estacion_origen',
       'domicilio_estacion_origen', 'duracion_recorrido',
       'fecha_destino_recorrido', 'id_estacion_destino',
       'nombre_estacion_destino', 'long_estacion_destino',
       'lat_estacion_destino', 'domicilio_estacion_destino'],
      dtype='object')

File: recorridos-realizados-2012.csv	 Columns:9
Index(['periodo', 'fecha_origen_recorrido', 'i

De esta forma, ya tenemos definidos los nombres de las columnas a leer para cada archivo.
Continuamos exportándolo a un archivo JSON

In [33]:
with open('files_schema.json', 'w') as json_file:
    json.dump(columns_per_files, json_file)